1. Importações e definições de variáveis de ambiente

In [86]:
# 1. Configuração e Importações

from pathlib import Path
from dotenv import load_dotenv
import unicodedata
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_batch
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 80)

print("ETL RAW -> SILVER | SINISTROS PRF")


ETL RAW -> SILVER | SINISTROS PRF


In [87]:
# 1.1 Caminhos (Raw e Silver)

BASE_PATH = Path(os.getcwd()).parent.parent
DATA_LAYER_RAW_PATH = BASE_PATH / "data_layer" / "raw"
DATA_LAYER_SILVER_PATH = BASE_PATH / "data_layer" / "silver" / "data"

DATA_LAYER_SILVER_PATH.mkdir(parents=True, exist_ok=True)

# Procura automaticamente CSVs de 2024 e 2025 na pasta raw
RAW_FILES = sorted([p for p in DATA_LAYER_RAW_PATH.iterdir() if p.suffix.lower() == ".csv"])

print(f"Arquivos encontrados em raw: {len(RAW_FILES)}")
for p in RAW_FILES:
    print(" -", p)



Arquivos encontrados em raw: 2
 - /home/matheus-brant/Desktop/referencia/SBD2-Grupo-19-PRF/data_layer/raw/dados_brutos_2024.csv
 - /home/matheus-brant/Desktop/referencia/SBD2-Grupo-19-PRF/data_layer/raw/dados_brutos_2025.csv


In [88]:
# 1.2 Banco (env)

load_dotenv(BASE_PATH / ".env")

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "database": os.getenv("POSTGRES_DB"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"),
}

print("DB host:", DB_CONFIG["host"])
print("DB port:", DB_CONFIG["port"])
print("DB name:", DB_CONFIG["database"])


DB host: localhost
DB port: 5432
DB name: prf


In [ ]:
# 2.1 Normalização de texto (tirar espaços, padronizar nulos e evitar lixo)

NULL_LIKE = {
    "", " ", "null", "none", "nan", "na", "n/a", "(null)", "NoneType", "NaN", "NULL", "N/A"
}

def normalize_text_series(s: pd.Series) -> pd.Series:
    """
    - strip + colapsa espaços
    - troca valores tipo 'null', 'nan', '' por <NA>
    - mantém como string (nullable)
    """
    s = s.astype("string")

    # normaliza unicode e remove espaços extras
    s = s.map(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)
    s = s.str.strip()
    s = s.str.replace(r"\s+", " ", regex=True)

    # padroniza nulos
    s_lower = s.str.lower()
    s = s.mask(s_lower.isin(NULL_LIKE), pd.NA)

    return s


def safe_to_int(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s, errors="coerce").astype("Int64")


def safe_to_float(s: pd.Series) -> pd.Series:
    # aceita vírgula ou ponto
    s = s.astype("string").str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce").astype("Float64")

def validate_coordinates(lat: pd.Series, lon: pd.Series) -> tuple[pd.Series, pd.Series]:
    
    lat_valid = lat.where((lat.isna()) | ((lat >= -90) & (lat <= 90)), pd.NA)
    lon_valid = lon.where((lon.isna()) | ((lon >= -180) & (lon <= 180)), pd.NA)
    return lat_valid.astype("Float64"), lon_valid.astype("Float64")



In [90]:
# 2.2 Parse de horário (aceita HH:MM:SS e HH:MM)

def parse_time_series(s: pd.Series) -> pd.Series:
    s = s.astype("string")
    s = normalize_text_series(s)

    t1 = pd.to_datetime(s, format="%H:%M:%S", errors="coerce")
    t2 = pd.to_datetime(s, format="%H:%M", errors="coerce")

    t = t1.fillna(t2)
    return t.dt.time  # python datetime.time


In [91]:
# 2.3 Dia da semana -> número (Seg=0 ... Dom=6)

DIA_SEMANA_MAP = {
    "segunda-feira": 0,
    "terca-feira": 1,
    "terça-feira": 1,
    "quarta-feira": 2,
    "quinta-feira": 3,
    "sexta-feira": 4,
    "sabado": 5,
    "sábado": 5,
    "domingo": 6,
}

def normalize_day_name(x: str) -> str:
    if x is None or pd.isna(x):
        return None
    x = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii")
    x = x.strip().lower()
    return x

def map_dia_semana_num(s: pd.Series) -> pd.Series:
    s = normalize_text_series(s)
    s = s.map(normalize_day_name)
    return s.map(DIA_SEMANA_MAP).astype("Int64")


In [92]:
# 2.4 Padronizar sexo -> masculino | feminino | ignorado

def padronizar_sexo(s: pd.Series) -> pd.Series:
    s = normalize_text_series(s)

    def _map(x):
        if x is None or pd.isna(x):
            return pd.NA
        v = str(x).strip().lower()
        v = unicodedata.normalize("NFKD", v).encode("ascii", "ignore").decode("ascii")

        if v in {"m", "masc", "masculino"}:
            return "masculino"
        if v in {"f", "fem", "feminino"}:
            return "feminino"

        # tudo que não der pra confiar cai em ignorado
        if v in {"ignorado", "nao informado", "nao-informado", "não informado", "não-informado", "0"}:
            return "ignorado"

        return "ignorado"

    out = s.map(_map).astype("string")
    return out

# 2.X Padronizar estado_fisico -> ileso | leve | grave | obito | ignorado

def padronizar_estado_fisico(s: pd.Series) -> pd.Series:
    s = normalize_text_series(s)

    def _map(x):
        if x is None or pd.isna(x):
            return "ignorado"

        v = str(x).strip().lower()
        # remove acentos pra facilitar comparação
        v = unicodedata.normalize("NFKD", v).encode("ascii", "ignore").decode("ascii")

        # casos comuns
        if "obito" in v or "morto" in v:
            return "obito"

        if "grave" in v:
            return "grave"

        # "lesoes leves", "lesao leve", "leve"
        if "leve" in v:
            return "leve"

        if "ileso" in v or "sem ferimentos" in v:
            return "ileso"

        # tudo que não for confiável vai pra ignorado
        if v in {"ignorado", "nao informado", "nao-informado", "0"}:
            return "ignorado"

        return "ignorado"

    return s.map(_map).astype("string")

# 2.X Padronizar uso_solo -> caracteristicas_via (urbano | rural | ignorado)
def padronizar_uso_solo(s: pd.Series) -> pd.Series:
    s = normalize_text_series(s)

    def _map(x):
        if x is None or pd.isna(x):
            return "ignorado"

        v = str(x).strip().lower()
        v = unicodedata.normalize("NFKD", v).encode("ascii", "ignore").decode("ascii")

        if v == "sim":
            return "urbano"
        if v == "nao":
            return "rural"

        return "ignorado"

    return s.map(_map).astype("string")




In [93]:
# 2.5 Faixa etária (idade_condutor / idade) -> bins 0-9 ... 100+

def faixa_etaria_bins(idade_s: pd.Series) -> pd.Series:
    idade = pd.to_numeric(idade_s, errors="coerce")

    # regra simples: 0 ou negativo = desconhecido
    idade = idade.mask((idade <= 0) | (idade > 120), np.nan)

    bins = [-0.1, 9, 19, 29, 39, 49, 59, 69, 79, 89, 99, 10_000]
    labels = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80-89","90-99","100+"]

    faixa = pd.cut(idade, bins=bins, labels=labels)
    return faixa.astype("string")


In [94]:
# 2.6 Faixa de idade do veículo (ano_fabricacao_veiculo -> idade do veículo -> bins)

def faixa_idade_veiculo_bins(ano_fab_s: pd.Series, ano_ref_s: pd.Series) -> pd.Series:
    ano_fab = pd.to_numeric(ano_fab_s, errors="coerce")
    ano_ref = pd.to_numeric(ano_ref_s, errors="coerce")

    idade_veic = ano_ref - ano_fab
    idade_veic = idade_veic.mask((idade_veic < 0) | (idade_veic > 120), np.nan)

    bins = [-0.1, 4, 9, 14, 19, 29, 120]
    labels = ["0-4","5-9","10-14","15-19","20-29","30+"]

    faixa = pd.cut(idade_veic, bins=bins, labels=labels)
    return faixa.astype("string")


In [95]:
def load_raw_csvs(csv_paths: list[Path]) -> pd.DataFrame:
    dfs = []
    for p in csv_paths:
        df = pd.read_csv(
            p,
            sep=";",
            encoding="ISO-8859-1",
            low_memory=False,
            dtype=str,  # lê tudo como string para controlar conversões depois
        )
        df["__source_file"] = p.name
        dfs.append(df)

    df_all = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
    return df_all


print("\nCarregando dados Raw...")
df_raw = load_raw_csvs(RAW_FILES)

print(f"Carregado: {df_raw.shape[0]:,} linhas x {df_raw.shape[1]:,} colunas")
print("Colunas:", list(df_raw.columns))



Carregando dados Raw...
Carregado: 372,148 linhas x 36 colunas
Colunas: ['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude', 'regional', 'delegacia', 'uop', '__source_file']


In [ ]:
# 4.1 Pipeline principal

DDL_COLUMNS = [
    "ano_arquivo",
    "sinistro_id",
    "pessoa_id",
    "veiculo_id",
    "data_hora",
    "dia_semana_num",
    "uf",
    "municipio",
    "delegacia",
    "latitude",
    "longitude",
    "causa_acidente",
    "tipo_acidente",
    "classificacao_acidente",
    "fase_dia",
    "sentido_via",
    "condicao_meteorologica",
    "tipo_pista",
    "tracado_via",
    "caracteristicas_via",
    "tipo_envolvido",
    "estado_fisico",
    "faixa_etaria_condutor",
    "sexo_condutor",
    "tipo_veiculo",
    "faixa_idade_veiculo",
    # "created_at" -> no banco tem default NOW(), então não precisa vir no insert
]


def transformar_para_silver(df: pd.DataFrame) -> pd.DataFrame:
    print("\nINICIANDO TRANSFORM (RAW -> SILVER)")
    print(f"Shape inicial: {df.shape}")

    df = df.copy()

    # 1) Normalizar strings em todas as colunas
    print("1) Normalizando texto...")
    for col in df.columns:
        df[col] = normalize_text_series(df[col])

    # 2) Corrigir nome de coluna (condicao_metereologica -> condicao_meteorologica)
    if "condicao_metereologica" in df.columns:
        df = df.rename(columns={"condicao_metereologica": "condicao_meteorologica"})

    # 3) IDs (padrão do banco)
    print("2) Convertendo IDs...")
    if "id" in df.columns:
        df["sinistro_id"] = safe_to_int(df["id"])
    else:
        df["sinistro_id"] = pd.NA

    if "pesid" in df.columns:
        df["pessoa_id"] = safe_to_int(df["pesid"])
    else:
        df["pessoa_id"] = pd.NA

    if "id_veiculo" in df.columns:
        df["veiculo_id"] = safe_to_int(df["id_veiculo"])
    else:
        df["veiculo_id"] = pd.NA

    # 4) Data e hora
    print("3) Convertendo data e horário...")
    df["data_inversa_dt"] = pd.to_datetime(df.get("data_inversa"), format="%Y-%m-%d", errors="coerce")
    df["horario_time"] = parse_time_series(df.get("horario"))

    # 5) data_hora (timestamp completo)
    print("4) Criando data_hora...")
    df["ano_arquivo"] = df["data_inversa_dt"].dt.year.astype("Int64")

    # data_hora: if time is missing, use 00:00:00 (don't lose the row)
    hora_txt = df["horario_time"].astype("string").fillna("00:00:00")
    df["data_hora"] = pd.to_datetime(
        df["data_inversa_dt"].astype("string") + " " + hora_txt,
        errors="coerce"
        )

    # 6) ano_arquivo (ano do sinistro)
    df["ano_arquivo"] = df["data_hora"].dt.year.astype("Int64")

    # 7) dia_semana_num
    df["dia_semana_num"] = map_dia_semana_num(df.get("dia_semana"))

    # 8) Latitude / Longitude
    print("5) Convertendo latitude/longitude...")
    df["latitude"] = safe_to_float(df.get("latitude"))
    df["longitude"] = safe_to_float(df.get("longitude"))
    df["latitude"], df["longitude"] = validate_coordinates(df["latitude"], df["longitude"])

    # 9) caracteristicas_via (vem de uso_solo) -> urbano | rural | ignorado
    df["caracteristicas_via"] = padronizar_uso_solo(df.get("uso_solo"))

    # 10) sexo_condutor (vem de sexo)
    df["sexo_condutor"] = padronizar_sexo(df.get("sexo"))

    # 10.5) estado_fisico padronizado (vem de estado_fisico)
    df["estado_fisico"] = padronizar_estado_fisico(df.get("estado_fisico"))

    # 11) faixa_etaria_condutor (vem de idade_condutor / idade)
    # no seu CSV está como "idade"
    df["faixa_etaria_condutor"] = faixa_etaria_bins(df.get("idade"))

    # 12) faixa_idade_veiculo (derivada de ano_fabricacao_veiculo)
    df["faixa_idade_veiculo"] = faixa_idade_veiculo_bins(df.get("ano_fabricacao_veiculo"), df["ano_arquivo"])

    # 13) UF em 2 letras maiúsculas
    df["uf"] = df.get("uf").str.upper()
    df["municipio"] = df.get("municipio").str.upper()


    # 14) Filtro do DDL: apenas 2024 ou 2025
    print("6) Aplicando filtro ano_arquivo (2024/2025)...")
    before = len(df)
    df = df[df["ano_arquivo"].isin([2024, 2025])].copy()
    print(f"   Linhas removidas pelo filtro: {before - len(df):,}")

    # 15) Ajustar colunas finais (DDL) + criar as que faltam como NULL
    print("7) Garantindo colunas do DDL...")
    for col in DDL_COLUMNS:
        if col not in df.columns:
            df[col] = pd.NA

    # 16) Remover duplicatas na PK (sinistro_id, pessoa_id)
    print("8) Removendo duplicatas por (sinistro_id, pessoa_id)...")
    before = len(df)
    df = df.sort_values(["sinistro_id", "pessoa_id", "data_hora"], na_position="last")
    df = df.drop_duplicates(subset=["sinistro_id", "pessoa_id"], keep="first")
    print(f"   Duplicatas removidas: {before - len(df):,}")

    # 17) Selecionar só as colunas do contrato (DDL)
    df_silver = df[DDL_COLUMNS].copy()

    print(f"Shape final (silver): {df_silver.shape}")
    return df_silver


df_silver = transformar_para_silver(df_raw)
df_silver.head()



INICIANDO TRANSFORM (RAW -> SILVER)
Shape inicial: (372148, 36)
1) Normalizando texto...
2) Convertendo IDs...
3) Convertendo data e horário...
4) Criando data_hora...
5) Convertendo latitude/longitude...
6) Aplicando filtro ano_arquivo (2024/2025)...
   Linhas removidas pelo filtro: 0
7) Garantindo colunas do DDL...
8) Removendo duplicatas por (sinistro_id, pessoa_id)...
   Duplicatas removidas: 12,141
Shape final (silver): (360007, 26)


,ano_arquivo,sinistro_id,pessoa_id,veiculo_id,data_hora,dia_semana_num,uf,municipio,delegacia,latitude,longitude,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_meteorologica,tipo_pista,tracado_via,caracteristicas_via,tipo_envolvido,estado_fisico,faixa_etaria_condutor,sexo_condutor,tipo_veiculo,faixa_idade_veiculo
41640,2024,571772,1268971,1018215,2024-01-01 00:05:00,0,RJ,TANGUA,DEL02-RJ,-22.72936,-42.701125,Reação tardia ou ineficiente do condutor,Colisão com objeto,Com Vítimas Fatais,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,urbano,Condutor,obito,20-29,masculino,Motocicleta,0-4
41641,2024,571774,1268985,1018226,2024-01-01 00:05:00,0,GO,ANAPOLIS,DEL02-GO,-16.229185,-49.009797,Velocidade Incompatível,Colisão com objeto,Sem Vítimas,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,rural,Condutor,ileso,30-39,feminino,Automóvel,15-19
41642,2024,571777,1269020,1018251,2024-01-01 01:45:00,0,ES,SERRA,DEL02-ES,-20.172928,-40.267364,Reação tardia ou ineficiente do condutor,Colisão com objeto,Sem Vítimas,Plena Noite,Decrescente,Nublado,Múltipla,Interseção de Vias;Reta,urbano,Condutor,ileso,50-59,masculino,Caminhonete,15-19
41643,2024,571778,1269028,1018261,2024-01-01 00:45:00,0,SC,PENHA,DEL03-SC,-26.83477,-48.706151,Acumulo de água sobre o pavimento,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Chuva,Dupla,Curva,rural,Condutor,ileso,50-59,masculino,Camioneta,10-14
41644,2024,571778,1269045,1018261,2024-01-01 00:45:00,0,SC,PENHA,DEL03-SC,-26.83477,-48.706151,Acumulo de água sobre o pavimento,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Chuva,Dupla,Curva,rural,Passageiro,leve,30-39,feminino,Camioneta,10-14


In [97]:
print("\nVALIDACOES")

print("Ano (value_counts):")
print(df_silver["ano_arquivo"].value_counts(dropna=False))

print("\nNulos nas chaves:")
print("sinistro_id null:", df_silver["sinistro_id"].isna().sum())
print("pessoa_id   null:", df_silver["pessoa_id"].isna().sum())

print("\nExemplo de colunas:")
print(df_silver[["sinistro_id","pessoa_id","veiculo_id","data_hora","dia_semana_num","sexo_condutor","faixa_etaria_condutor"]].head(10))



VALIDACOES
Ano (value_counts):
ano_arquivo
2024    189998
2025    170009
Name: count, dtype: Int64

Nulos nas chaves:
sinistro_id null: 0
pessoa_id   null: 0

Exemplo de colunas:
       sinistro_id  pessoa_id  veiculo_id           data_hora  dia_semana_num  \
41640       571772    1268971     1018215 2024-01-01 00:05:00               0   
41641       571774    1268985     1018226 2024-01-01 00:05:00               0   
41642       571777    1269020     1018251 2024-01-01 01:45:00               0   
41643       571778    1269028     1018261 2024-01-01 00:45:00               0   
41644       571778    1269045     1018261 2024-01-01 00:45:00               0   
41648       571779    1268976     1018219 2024-01-01 01:45:00               0   
41645       571779    1268998     1018219 2024-01-01 01:45:00               0   
41646       571779    1268999     1018219 2024-01-01 01:45:00               0   
41647       571779    1269000     1018219 2024-01-01 01:45:00               0   
41650     

In [98]:
OUTPUT_FILE = DATA_LAYER_SILVER_PATH / "sinistros_silver.csv"

df_silver.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
print("CSV Silver salvo em:", OUTPUT_FILE)
print("Tamanho (linhas):", len(df_silver))


CSV Silver salvo em: /home/matheus-brant/Desktop/referencia/SBD2-Grupo-19-PRF/data_layer/silver/data/sinistros_silver.csv
Tamanho (linhas): 360007


In [ ]:
print("\nQuick checks:")
print("latitude null %:", df_silver["latitude"].isna().mean())
print("longitude null %:", df_silver["longitude"].isna().mean())
print("ano_arquivo null:", df_silver["ano_arquivo"].isna().sum())
print("data_hora null:", df_silver["data_hora"].isna().sum())
print("municipio sample:", df_silver["municipio"].dropna().unique()[:5])
